In [0]:
!pip install mlflow

Collecting mlflow
 Downloading mlflow-1.20.1-py3-none-any.whl (14.6 MB)
 | 10 kB 25.3 MB/s eta 0:00:01 | | 20 kB 21.2 MB/s eta 0:00:01 | | 30 kB 11.2 MB/s eta 0:00:02 | | 40 kB 6.5 MB/s eta 0:00:03 |▏ | 51 kB 7.5 MB/s eta 0:00:02 |▏ | 61 kB 7.6 MB/s eta 0:00:02 |▏ | 71 kB 6.6 MB/s eta 0:00:03 |▏ | 81 kB 7.4 MB/s eta 0:00:02 |▏ | 92 kB 6.7 MB/s eta 0:00:03 |▎ | 102 kB 5.7 MB/s eta 0:00:03 |▎ | 112 kB 5.7 MB/s eta 0:00:03 |▎ | 122 kB 5.7 MB/s eta 0:00:03 |▎ | 133 kB 5.7 MB/s eta 0:00:03 |▎ | 143 kB 5.7 MB/s eta 0:00:03 |▍ | 153 kB 5.7 MB/s eta 0:00:03 |▍ | 163 kB 5.7 MB/s eta 0:00:03 |▍ | 174 kB 5.7 MB/s eta 0:00:03 |▍ | 184 kB 5.7 MB/s eta 0:00:03 |▍ | 194 kB 5.7 MB/s eta 0:00:03 |▌ | 204 kB 5.7 MB/s eta 0:00:03 |▌ | 215 kB 5.7 MB/s eta 0:00:03 |▌ | 225 kB 5.7 MB/s eta 0:00:03 |▌ | 235 kB 5.7 MB/s eta 0:00:03 |▌ | 245 kB 5.7 MB/s eta 0:00:03 |▋ | 256 kB 5.7 MB/s eta 0:00:03 |▋ | 266 kB 5.7 MB/s eta 0:00:03 |▋ | 276 kB 5.7 MB/s eta 0:00:03 |▋ | 286 kB 5.7 MB/s eta 0:00:03 |▋ | 296 kB 5.7 MB/s eta 0:00:03 |▊ | 307 kB 5.7 MB/s eta 0:00:03 |▊ | 317 kB 5.7 MB/s eta 0:00:03 |▊ | 327 kB 5.7 MB/s eta 0:00:03 |▊ | 337 kB 5.7 MB/s eta 0:00:03 |▊ | 348 kB 5.7 MB/s eta 0:00:03 |▉ | 358 kB 5.7 MB/s eta 0:00:03 |▉ | 368 kB 5.7 MB/s eta 0:00:03 |▉ | 378 kB 5.7 MB/s eta 0:00:03 |▉ | 389 kB 5.7 MB/s eta 0:00:03 |▉ | 399 kB 5.7 MB/s eta 0:00:03 |█ | 409 kB 5.7 MB/s eta 0:00:03 |█ | 419 kB 5.7 MB/s eta 0:00:03 |█ | 430 kB 5.7 MB/s eta 0:00:03 |█ | 440 kB 5.7 MB/s eta 0:00:03 |█ | 450 kB 5.7 MB/s eta 0:00:03 |█ | 460 kB 5.7 MB/s eta 0:00:03 |█ | 471 kB 5.7 MB/s eta 0:00:03 |█ | 481 kB 5.7 MB/s eta 0:00:03 |█ | 491 kB 5.7 MB/s eta 0:00:03 |█ | 501 kB 5.7 MB/s eta 0:00:03 |█▏ | 512 kB 5.7 MB/s eta 0:00:03 |█▏ | 522 kB 5.7 MB/s eta 0:00:03 |█▏ | 532 kB 5.7 MB/s eta 0:00:03 |█▏ | 542 kB 5.7 MB/s eta 0:00:03 |█▏ | 552 kB 5.7 MB/s eta 0:00:03 |█▎ | 563 kB 5.7 MB/s eta 0:00:03 |█▎ | 573 kB 5.7 MB/s eta 0:00:03 |█▎ | 583 kB 5.7 MB/s eta 0:00:03 |█▎ | 593 kB 5.7 MB/s eta 0:00:03 |█▎ | 604 kB 5.7 MB/s eta 0:00:03 |█▍ | 614 kB 5.7 MB/s eta 0:00:03 |█▍ | 624 kB 5.7 MB/s eta 0:00:03 |█▍ | 634 kB 5.7 MB/s eta 0:00:03 |█▍ | 645 kB 5.7 MB/s eta 0:00:03 |█▍ | 655 kB 5.7 MB/s eta 0:00:03 |█▌ | 665 kB 5.7 MB/s eta 0:00:03 |█▌ | 675 kB 5.7 MB/s eta 0:00:03 |█▌ | 686 kB 5.7 MB/s eta 0:00:03 |█▌ | 696 kB 5.7 MB/s eta 0:00:03 |█▌ | 706 kB 5.7 MB/s eta 0:00:03 |█▋ | 716 kB 5.7 MB/s eta 0:00:03 |█▋ | 727 kB 5.7 MB/s eta 0:00:03 |█▋ | 737 kB 5.7 MB/s eta 0:00:03 |█▋ | 747 kB 5.7 MB/s eta 0:00:03 |█▋ | 757 kB 5.7 MB/s eta 0:00:03 |█▊ | 768 kB 5.7 MB/s eta 0:00:03 |█▊ | 778 kB 5.7 MB/s eta 0:00:03 |█▊ | 788 kB 5.7 MB/s eta 0:00:03 |█▊ | 798 kB 5.7 MB/s eta 0:00:03 |█▊ | 808 kB 5.7 MB/s eta 0:00:03 |█▉ | 819 kB 5.7 MB/s eta 0:00:03 |█▉ | 829 kB 5.7 MB/s eta 0:00:03 |█▉ | 839 kB 5.7 MB/s eta 0:00:03 |█▉ | 849 kB 5.7 MB/s eta 0:00:03 |█▉ | 860 kB 5.7 MB/s eta 0:00:03 |██ | 870 kB 5.7 MB/s eta 0:00:03 |██ | 880 kB 5.7 MB/s eta 0:00:03 |██ | 890 kB 5.7 MB/s eta 0:00:03 |██ | 901 kB 5.7 MB/s eta 0:00:03 |██ | 911 kB 5.7 MB/s eta 0:00:03 |██ | 921 kB 5.7 MB/s eta 0:00:03 |██ | 931 kB 5.7 MB/s eta 0:00:03 |██ | 942 kB 5.7 MB/s eta 0:00:03 |██ | 952 kB 5.7 MB/s eta 0:00:03 |██ | 962 kB 5.7 MB/s eta 0:00:03 |██▏ | 972 kB 5.7 MB/s eta 0:00:03 |██▏ | 983 kB 5.7 MB/s eta 0:00:03 |██▏ | 993 kB 5.7 MB/s eta 0:00:03 |██▏ | 1.0 MB 5.7 MB/s eta 0:00:03 |██▏ | 1.0 MB 5.7 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.7 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.7 MB/s eta 0:00:03 |██▎ | 1.0 MB 5.7 MB/s eta 0:00:03 |██▎ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▎ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▍ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▌ | 1.1 MB 5.7 MB/s eta 0:00:03 |██▌ | 1.2 MB 5.7 MB/s eta 0:00:03 |██▌ | 1.2 M

In [0]:
!pip install hyperopt

Collecting hyperopt
 Downloading hyperopt-0.2.5-py2.py3-none-any.whl (965 kB)
▍ | 10 kB 16.7 MB/s eta 0:00:01 |▊ | 20 kB 11.8 MB/s eta 0:00:01 |█ | 30 kB 14.8 MB/s eta 0:00:01 |█▍ | 40 kB 17.4 MB/s eta 0:00:01 |█▊ | 51 kB 8.5 MB/s eta 0:00:01 |██ | 61 kB 7.1 MB/s eta 0:00:01 |██▍ | 71 kB 7.1 MB/s eta 0:00:01 |██▊ | 81 kB 8.0 MB/s eta 0:00:01 |███ | 92 kB 8.4 MB/s eta 0:00:01 |███▍ | 102 kB 6.6 MB/s eta 0:00:01 |███▊ | 112 kB 6.6 MB/s eta 0:00:01 |████ | 122 kB 6.6 MB/s eta 0:00:01 |████▍ | 133 kB 6.6 MB/s eta 0:00:01 |████▊ | 143 kB 6.6 MB/s eta 0:00:01 |█████ | 153 kB 6.6 MB/s eta 0:00:01 |█████▍ | 163 kB 6.6 MB/s eta 0:00:01 |█████▊ | 174 kB 6.6 MB/s eta 0:00:01 |██████ | 184 kB 6.6 MB/s eta 0:00:01 |██████▌ | 194 kB 6.6 MB/s eta 0:00:01 |██████▉ | 204 kB 6.6 MB/s eta 0:00:01 |███████▏ | 215 kB 6.6 MB/s eta 0:00:01 |███████▌ | 225 kB 6.6 MB/s eta 0:00:01 |███████▉ | 235 kB 6.6 MB/s eta 0:00:01 |████████▏ | 245 kB 6.6 MB/s eta 0:00:01 |████████▌ | 256 kB 6.6 MB/s eta 0:00:01 |████████▉ | 266 kB 6.6 MB/s eta 0:00:01 |█████████▏ | 276 kB 6.6 MB/s eta 0:00:01 |█████████▌ | 286 kB 6.6 MB/s eta 0:00:01 |█████████▉ | 296 kB 6.6 MB/s eta 0:00:01 |██████████▏ | 307 kB 6.6 MB/s eta 0:00:01 |██████████▌ | 317 kB 6.6 MB/s eta 0:00:01 |██████████▉ | 327 kB 6.6 MB/s eta 0:00:01 |███████████▏ | 337 kB 6.6 MB/s eta 0:00:01 |███████████▌ | 348 kB 6.6 MB/s eta 0:00:01 |███████████▉ | 358 kB 6.6 MB/s eta 0:00:01 |████████████▏ | 368 kB 6.6 MB/s eta 0:00:01 |████████████▌ | 378 kB 6.6 MB/s eta 0:00:01 |█████████████ | 389 kB 6.6 MB/s eta 0:00:01 |█████████████▎ | 399 kB 6.6 MB/s eta 0:00:01 |█████████████▋ | 409 kB 6.6 MB/s eta 0:00:01 |██████████████ | 419 kB 6.6 MB/s eta 0:00:01 |██████████████▎ | 430 kB 6.6 MB/s eta 0:00:01 |██████████████▋ | 440 kB 6.6 MB/s eta 0:00:01 |███████████████ | 450 kB 6.6 MB/s eta 0:00:01 |███████████████▎ | 460 kB 6.6 MB/s eta 0:00:01 |███████████████▋ | 471 kB 6.6 MB/s eta 0:00:01 |████████████████ | 481 kB 6.6 MB/s eta 0:00:01 |████████████████▎ | 491 kB 6.6 MB/s eta 0:00:01 |████████████████▋ | 501 kB 6.6 MB/s eta 0:00:01 |█████████████████ | 512 kB 6.6 MB/s eta 0:00:01 |█████████████████▎ | 522 kB 6.6 MB/s eta 0:00:01 |█████████████████▋ | 532 kB 6.6 MB/s eta 0:00:01 |██████████████████ | 542 kB 6.6 MB/s eta 0:00:01 |██████████████████▎ | 552 kB 6.6 MB/s eta 0:00:01 |██████████████████▋ | 563 kB 6.6 MB/s eta 0:00:01 |███████████████████ | 573 kB 6.6 MB/s eta 0:00:01 |███████████████████▍ | 583 kB 6.6 MB/s eta 0:00:01 |███████████████████▊ | 593 kB 6.6 MB/s eta 0:00:01 |████████████████████ | 604 kB 6.6 MB/s eta 0:00:01 |████████████████████▍ | 614 kB 6.6 MB/s eta 0:00:01 |████████████████████▊ | 624 kB 6.6 MB/s eta 0:00:01 |█████████████████████ | 634 kB 6.6 MB/s eta 0:00:01 |█████████████████████▍ | 645 kB 6.6 MB/s eta 0:00:01 |█████████████████████▊ | 655 kB 6.6 MB/s eta 0:00:01 |██████████████████████ | 665 kB 6.6 MB/s eta 0:00:01 |██████████████████████▍ | 675 kB 6.6 MB/s eta 0:00:01 |██████████████████████▊ | 686 kB 6.6 MB/s eta 0:00:01 |███████████████████████ | 696 kB 6.6 MB/s eta 0:00:01 |███████████████████████▍ | 706 kB 6.6 MB/s eta 0:00:01 |███████████████████████▊ | 716 kB 6.6 MB/s eta 0:00:01 |████████████████████████ | 727 kB 6.6 MB/s eta 0:00:01 |████████████████████████▍ | 737 kB 6.6 MB/s eta 0:00:01 |████████████████████████▊ | 747 kB 6.6 MB/s eta 0:00:01 |█████████████████████████ | 757 kB 6.6 MB/s eta 0:00:01 |█████████████████████████▍ | 768 kB 6.6 MB/s eta 0:00:01 |█████████████████████████▉ | 778 kB 6.6 MB/s eta 0:00:01 |██████████████████████████▏ | 788 kB 6.6 MB/s eta 0:00:01 |██████████████████████████▌ | 798 kB 6.6 MB/s eta 0:00:01 |██████████████████████████▉ | 808 kB 6.6 MB/s eta 0:00:01 |███████████████████████████▏ | 819 kB 6.6 MB/s eta 0:00:01 |███████████████████████████▌ | 829 kB 6.6 MB/s eta 0:00:01 |███████████████████████████▉ | 839 kB 6.6 MB/s eta 0:00:01 |████████████████████████████▏ | 84

In [0]:
#Import Libraries
import mlflow
import mlflow.spark
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from hyperopt import fmin, tpe, hp, SparkTrials, Trials, STATUS_OK
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession

In [0]:
# Note: On environments like Databricks with pre-created SparkSessions,
# ensure the org.mlflow:mlflow-spark:1.11.0 is attached as a library to
# your cluster
spark = (SparkSession.builder
            .config("spark.jars.packages", "org.mlflow:mlflow-spark:1.11.0")
            .master("local[*]")
            .getOrCreate())

##Basic EDA

In [0]:
#Read parquet 
data = spark.read.parquet("/tmp/output/higgsdf.parquet")

In [0]:
#Check data
data.show(5)

+-----+------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+--------------------+-------------------+-----------------+-------------------+--------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
label| feature1| feature2| feature3| feature4| feature5| feature6| feature7| feature8| feature9| feature10| feature11| feature12| feature13| feature14| feature15| feature16| feature17| feature18| feature19| feature20| feature21| feature22| feature23| feature24| feature25| feature26| feature27| feature28|
+-----+------------------+-------------------+-------------------+------------------+-------------------+------------------+--------------------+-------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+--------------------+-------------------+-----------------+-------------------+--------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
 1.0| 0.869293212890625|-0.6350818276405334|0.22569026052951813| 0.327470064163208|-0.6899932026863098|0.7542022466659546|-0.24857313930988312|-1.0920639038085938| 0.0|1.3749921321868896|-0.6536741852760315| 0.9303491115570068|1.1074360609054565| 1.138904333114624| -1.5781983137130737| -1.046985387802124| 0.0| 0.657929539680481|-0.01045456994324...| -0.0457671694457531|3.101961374282837| 1.353760004043579|0.9795631170272827| 0.978076159954071|0.9200048446655273|0.7216574549674988|0.9887509346008301|0.8766783475875854|
 1.0|0.9075421094894409| 0.3291472792625427| 0.3594118654727936|1.4979698657989502|-0.3130095303058624| 1.09553062915802| -0.5575249195098877| -1.588229775428772|2.1730761528015137|0.8125811815261841|-0.2136419266462326| 1.2710145711898804| 2.214872121810913|0.4999939501285553| -1.2614318132400513| 0.7321561574935913| 0.0|0.39870089292526245| -1.138930082321167|-8.19110195152461...| 0.0|0.3022198975086212|0.8330481648445129|0.9856996536254883|0.9780983924865723|0.7797321677207947|0.9923557639122009|0.7983425855636597|
 1.0|0.7988347411155701| 1.4706387519836426|-1.6359747648239136| 0.45377317070961| 0.4256291687488556| 1.104874610900879| 1.2823222875595093| 1.3816642761230469| 0.0|0.8517372012138367| 1.540658950805664|-0.8196895122528076| 2.214872121810913|0.9934899210929871| 0.3560801148414612|-0.2087775468826294|2.548224449157715| 1.256954550743103| 1.128847599029541| 0.9004608392715454| 0.0|0.9097532629966736| 1.108330488204956|0.9856922030448914|0.9513312578201294|0.8032515048980713|0.8659244179725647|0.7801175713539124|
 0.0| 1.344384789466858|-0.8766260147094727| 0.9359127283096313| 1.992050051689148| 0.8824543952941895| 1.786065936088562| -1.6467777490615845|-0.9423825144767761| 0.0| 2.423264741897583|-0.6760157942771912| 0.7361586689949036| 2.214872121810913|1.2987197637557983| -1.430738091468811|-0.3646581768989563| 0.0| 0.7453126907348633| -0.6783788204193115| -1.360356330871582| 0.0|0.9466524720191956|1.0287036895751953|0.9986560940742493|0.7282806038856506|0.8692002296447754|1.0267364978790283|0.9579039812088013|
 1.0|1.1050089597702026|0.32135555148124695| 1.522401213645935|0.8828076124191284| -1.205349326133728|0.6814661026000977| -1.0704638957977295|-0.9218706488609314| 0.0|0.8008721470832825| 1.020974040031433| 0.9714065194129944| 2.214872121810913|0.5967612862586975|-0.35027286410331726| 0.6311942934989929| 0.0| 0.4799988865852356|-0.37356552481651306| 0.11304061114788055| 0.0| 0.755856454372406|1.3610570430755615| 0.986609697341919|0.8380846381187439|1.1332951784133911|0.8722448945045471|0.8084865212440491|
+

In [0]:
#Check the label distribution
labelcounts = data.groupBy('label').count().show()

+-----+-------+
label| count|
+-----+-------+
 0.0|5170877|
 1.0|5829123|
+-----+-------+

In [0]:
# import sql function pyspark
import pyspark.sql.functions as f

# null values in each column
data_agg = data.agg(*[f.count(f.when(f.isnull(c), c)).alias(c) for c in data.columns])
data_agg.show()

+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
label|feature1|feature2|feature3|feature4|feature5|feature6|feature7|feature8|feature9|feature10|feature11|feature12|feature13|feature14|feature15|feature16|feature17|feature18|feature19|feature20|feature21|feature22|feature23|feature24|feature25|feature26|feature27|feature28|
+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+

In [0]:
features =['feature1','feature2','feature3','feature4','feature5','feature6','feature7','feature8','feature9','feature10','feature11','feature12','feature13','feature14','feature15','feature16','feature17','feature18','feature19','feature20','feature21','feature22','feature23','feature24','feature25','feature26','feature27','feature28']

##Vanilla Pipeline

In [0]:
stages = [] #stages in pipeline

In [0]:
#Start by using vector assembler to truncate all the columns
vecAssembler = VectorAssembler(inputCols = features, outputCol="features")
stages += [vecAssembler]

In [0]:
#Check to see vector assembler output

testdf = vecAssembler.transform(data)
testdf.head().features

Out[36]: DenseVector([0.8693, -0.6351, 0.2257, 0.3275, -0.69, 0.7542, -0.2486, -1.0921, 0.0, 1.375, -0.6537, 0.9303, 1.1074, 1.1389, -1.5782, -1.047, 0.0, 0.6579, -0.0105, -0.0458, 3.102, 1.3538, 0.9796, 0.9781, 0.92, 0.7217, 0.9888, 0.8767])

In [0]:
#Scale the data
scaler = MinMaxScaler( min=-1.0, max=1.0,inputCol="features", outputCol="scaledFeatures")
stages += [scaler]
stages

Out[48]: [VectorAssembler_95062a0eba2e, MinMaxScaler_696663cb36ce]

In [0]:
#Test the scaler
# Compute summary statistics and generate MinMaxScalerModel
scalerModel = scaler.fit(testdf)

# rescale each feature to range [min, max].
testdf2 = scalerModel.transform(testdf)

In [0]:
#Build baseline classifier
lr = LogisticRegression(labelCol="label",
                        featuresCol="features",
                        maxIter=10,
                        regParam=0.3)

stages += [lr]

In [0]:
#Split train and test label
train,test = data.randomSplit([0.8, 0.2])

In [0]:
#Check the ratio 
train.groupBy('label').count().show()

+-----+-------+
label| count|
+-----+-------+
 0.0|4137657|
 1.0|4661460|
+-----+-------+

In [0]:
# Enable Spark datasource autologging.
try:
  import mlflow.pyspark.ml
  mlflow.pyspark.ml.autolog()
except:
  print(f"Your version of MLflow ({mlflow.__version__}) does not support pyspark.ml for autologging. To use autologging, upgrade your MLflow client version or use Databricks Runtime for ML 8.3 or above.")

2021/09/02 02:58:59 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable

In [0]:


#Build the pipeline
with mlflow.start_run(run_name='logreg_baseline') as run:
  Regpipeline = Pipeline(stages = stages) 
  
  model = Regpipeline.fit(train)

  validation_metric = evaluator.evaluate(predictions)
  mlflow.log_metric("val_f1_score", validation_metric)

2021/09/02 02:58:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable
2021/09/02 02:58:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable
2021/09/02 02:58:48 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during spark autologging: Exception while attempting to initialize JVM-side state for Spark datasource autologging. Please create a new Spark session and ensure you have the mlflow-spark JAR attached to your Spark session as described in http://mlflow.org/docs/latest/tracking.html#automatic-logging-from-spark-experimental. Exception:
'JavaPackage' object is not callable

--------------------------------------------------------------------------- 
 InvalidConfigurationError Traceback (most recent call last)
 <command-3753100686505342> in <module> 
 7 
 8 #Build the pipeline 
 ----> 9 with mlflow . start_run ( run_name = 'logreg_baseline' ) as run : 
 10 Regpipeline = Pipeline ( stages = stages ) 
 11 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/fluent.py in start_run (run_id, experiment_id, run_name, nested, tags) 
 247 tags = context_registry . resolve_tags ( user_specified_tags ) 
 248 
 --> 249 active_run_obj = client . create_run ( experiment_id = exp_id_for_run , tags = tags ) 
 250 
 251 _active_run_stack . append ( ActiveRun ( active_run_obj ) ) 

 /databricks/python_shell/dbruntime/MlflowCreateRunHook.py in patched_create_run (*args, **kwargs) 
 32 
 33 def patched_create_run ( * args , ** kwargs ) : 
 ---> 34 res = orig_create_run ( * args , ** kwargs ) 
 35 try : 
 36 self . increment_runs_created ( res . info . experiment_id ) 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/client.py in create_run (self, experiment_id, start_time, tags) 
 263 status : RUNNING
 264 """
 --> 265 return self . _tracking_client . create_run ( experiment_id , start_time , tags ) 
 266 
 267 def list_run_infos(

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_tracking_service/client.py in create_run (self, experiment_id, start_time, tags) 
 98 user_id = tags . get ( MLFLOW_USER , "unknown" ) 
 99 
 --> 100 return self.store.create_run(
 101 experiment_id = experiment_id , 
 102 user_id = user_id , 

 /databricks/python/lib/python3.8/site-packages/mlflow/store/tracking/rest_store.py in create_run (self, experiment_id, user_id, start_time, tags) 
 161 )
 162 )
 --> 163 response_proto = self . _call_endpoint ( CreateRun , req_body ) 
 164 run = Run . from_proto ( response_proto . run ) 
 165 return run

 /databricks/python/lib/python3.8/site-packages/mlflow/store/tracking/rest_store.py in _call_endpoint (self, api, json_body) 
 54 endpoint , method = _METHOD_TO_INFO [ api ] 
 55 response_proto = api . Response ( ) 
 ---> 56 return call_endpoint ( self . get_host_creds ( ) , endpoint , method , json_body , response_proto ) 
 57 
 58 def list_experiments(

 /databricks/python/lib/python3.8/site-packages/mlflow/utils/databricks_utils.py in get_databricks_host_creds (server_uri) 
 310 config = provider . ProfileConfigProvider ( profile ) . get_config ( ) 
 311 else : 
 --> 312 config = provider . get_config ( ) 
 313 # if a path is specified, that implies a Databricks tracking URI of the form: 
 314 # databricks://profile-name/path-specifier 

 /databricks/python/lib/python3.8/site-packages/databricks_cli/configure/provider.py in get_config () 
 120 if config : 
 121 return config
 --> 122 raise InvalidConfigurationError . for_profile ( None ) 
 123 
 124 

 InvalidConfigurationError : You haven't configured the CLI yet! Please configure by entering `/databricks/python_shell/scripts/PythonShell.py configure`

In [0]:
predicted = model.transform(train)

In [0]:
predicted.select('features', 'label', 'rawPrediction', 'probability', 'prediction').show()

+--------------------+-----+--------------------+--------------------+----------+
 features|label| rawPrediction| probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
[0.27469664812088...| 0.0|[0.00843005004345...|[0.50210750002995...| 0.0|
[0.27469664812088...| 0.0|[-0.1656073204684...|[0.45869253422925...| 1.0|
[0.27469664812088...| 0.0|[-0.2878721826824...|[0.42852487159626...| 1.0|
[0.27469664812088...| 0.0|[-0.0964503119186...|[0.47590609724584...| 1.0|
[0.27469664812088...| 0.0|[-0.0085673520800...|[0.49785817508070...| 1.0|
[0.27469664812088...| 0.0|[0.08564011149672...|[0.52139695196086...| 0.0|
[0.27469664812088...| 0.0|[0.43593343831387...|[0.60728962506287...| 0.0|
[0.27469664812088...| 0.0|[-0.0101214056505...|[0.49746967018850...| 1.0|
[0.27469664812088...| 0.0|[0.69056074772823...|[0.66609165626476...| 0.0|
[0.27469664812088...| 0.0|[-0.1123163375238...|[0.47195039647271...| 1.0|
[0.27469664812088...| 0.0|[-0.2359698610026...|[0.44127975285720...| 1.0|
[0.27469664812088...| 0.0|[-0.1421038994497...|[0.46453368752953...| 1.0|
[0.27469664812088...| 0.0|[0.09706939528682...|[0.52424831190018...| 0.0|
[0.27469664812088...| 0.0|[0.16647912747316...|[0.54152392214299...| 0.0|
[0.27469664812088...| 0.0|[0.02653745974610...|[0.50663397561696...| 0.0|
[0.27469664812088...| 0.0|[0.28872546499154...|[0.57168407706667...| 0.0|
[0.27469664812088...| 0.0|[-0.1527891444987...|[0.46187684893304...| 1.0|
[0.27469664812088...| 0.0|[0.20188310832158...|[0.55030005395649...| 0.0|
[0.27469664812088...| 0.0|[-0.0615435902059...|[0.48461895692925...| 1.0|
[0.27469664812088...| 0.0|[-0.1922690457407...|[0.45208026995424...| 1.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 20 rows